# **数据清洗和准备**
---


## **处理缺失值**

> **_None值也属于NaN_**   
> **_当进行数据清洗以进行分析时，最好直接对缺失数据进行分析，以判断数据采集的问题或缺失数据可能导致的偏差。_**

In [2]:
import numpy as np 
import pandas as pd
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

> **NaN值处理方式**  
> **dropna | 移除NaN值**   
> **fillna | 填充NaN值**  
> **isnull | 判断null值**   
> **notnull | 过滤null值** 


### **dropna()过滤缺失值**

> **Series的NaN值处理方式**

In [3]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [4]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

> **Dataframe的NaN值处理方式：how='all'只删除全部NAN值**

In [5]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                    [NA, NA, NA], [NA, 6.5, 3.]])

cleaned = data.dropna()

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [6]:
data.dropna(how='all') 

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [7]:
data[4] = NA
data.dropna(axis=1,how='all') # 传入axis=1 对列操作

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


> **保留至少有n个非NaN数据的行/列（用axis=0或1表示）**  
> **简单的理解：这一行除去NA值，剩余数值的数量大于等于n，便显示这一行。**  
> **df.dropna(thresh=2):留下一部分观测数据**

In [66]:
df = pd.DataFrame(np.random.randn(7, 3))

df.iloc[:4, 1] = NA

df.iloc[:2, 2] = NA

df

,0,1,2
0,0.533639,NaN,NaN
1,1.026165,NaN,NaN
2,-2.025905,NaN,-1.095058
3,0.541854,NaN,-0.587055
4,1.305447,-0.982238,0.846471
5,1.399004,-0.098724,-0.402551
6,1.409971,-0.570024,1.813019


In [73]:
df.dropna(thresh=3)

,0,1,2
4,1.305447,-0.982238,0.846471
5,1.399004,-0.098724,-0.402551
6,1.409971,-0.570024,1.813019


### **fillna()填充数据**

In [9]:
df.fillna(0)

,0,1,2
0,-0.881671,0.000000,0.000000
1,-0.394237,0.000000,0.000000
2,-0.738713,0.000000,-0.861347
3,-0.998645,0.000000,-0.096758
4,-1.064049,-1.863153,0.948280
5,-1.223059,0.620141,0.309420
6,-0.546256,-0.725746,-0.475331


In [10]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.881671,0.500000,0.000000
1,-0.394237,0.500000,0.000000
2,-0.738713,0.500000,-0.861347
3,-0.998645,0.500000,-0.096758
4,-1.064049,-1.863153,0.948280
5,-1.223059,0.620141,0.309420
6,-0.546256,-0.725746,-0.475331


> **reindexing有效的那些插值方法也可用于fillna**

In [11]:
df.fillna(method='ffill')
df.fillna(method='ffill', limit=2)

,0,1,2
0,-0.881671,NaN,NaN
1,-0.394237,NaN,NaN
2,-0.738713,NaN,-0.861347
3,-0.998645,NaN,-0.096758
4,-1.064049,-1.863153,0.948280
5,-1.223059,0.620141,0.309420
6,-0.546256,-0.725746,-0.475331


> **可以传入很多聚合函数**

In [12]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

> **fillna的参数**  
> **value:填充的值或者字典对象**  
> **method：插值方式，默认ffill**  
> **axis:轴**
> **inplace:在对象上修改，不产生副本**  
> **limit：连续填充的最大数量**

---
## **数据转换**

### **drop_duplicated()移除重复值**

> **df.duplicated()查找重复值**

In [13]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                       'k2': [1, 1, 2, 3, 3, 4, 4]})
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

> **df.drop_duplicates()删除重复值**

In [14]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [15]:
data['v1'] = range(7)
data.drop_duplicates(['k1']) # 根据固定列删除重复值
data.drop_duplicates(['k1', 'k2'], keep='last') # 默认保留最后一个重复值

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### **利用函数或映射转换数据**

> **S.map()接受函数或者映射关系的字典型对象**

In [16]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                                'Pastrami', 'corned beef', 'Bacon',
                                'pastrami', 'honey ham', 'nova lox'],
                                'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

meat_to_animal = {
                    'bacon': 'pig',
                    'pulled pork': 'pig',
                    'pastrami': 'cow',
                    'corned beef': 'cow',
                    'honey ham': 'pig',
                    'nova lox': 'salmon'
                    }

lowercased = data['food'].str.lower() # 字符串大写转小写
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


> **S.map()一次性搞定**

In [17]:
# (lambda x,y:x*y) 是匿名函数，常规函数的简写版
data['food'].map(lambda x:meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### **df.replace()替换值**

> **replace(x,y)替换值**  
> 笔记：data.replace方法与data.str.replace不同，后者做的是字符串的元素级替换。我们会在后面学习Series的字符串方法。

In [18]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

data


0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [19]:
data.replace(-999, np.nan)
data.replace([-999, -1000], np.nan) # 替换多个值
data.replace([-999, -1000], [np.nan,0]) # 替换成不同的值
data.replace({-999: np.nan, -1000: 0}) # 也可以传入字典

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### **df.rename()重命名轴索引**

> **index.map()通过函数更改轴索引名**

In [4]:
import numpy as np
import pandas as pd
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                     index=['Ohio', 'Colorado', 'New York'],
                     columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [5]:
transform = lambda x:x[:4].upper() # 注意这里把index列表的每个值拿出来取[:4]
data.index = data.index.map(transform)
data 

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


> **df.rename()重命名索引**

In [6]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [7]:
data.reindex(index=[1,2,3])

,one,two,three,four
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN


In [10]:
data.rename(index=str.lower,columns=str.upper)

,ONE,TWO,THREE,FOUR
ohio,0,1,2,3
colo,4,5,6,7
new,8,9,10,11


> **df.rename()用字典对象更改轴标签**

> **_df.rename()更改列名和索引名、不更改源数据_**  
> **_df.reindex()更改索引名、更改源数据并返回新索引_**  
> ![](img/rename%E5%92%8Creindex.jpg)

In [23]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'},inplace=True)
                                                # inplace=True 修改源数据
data

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### **cut() + qcut()离散化和面元划分——其实就是区间/数据桶的概念**

> **pd.cut(arg1,arg2)传入指定区间，根据样本指定区间划分**

In [24]:
import pandas as pd
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages,bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [25]:
cats.codes # 年龄数据标签

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [26]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [27]:
pd.value_counts(cats) # 计数值

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

> **right=False闭合方式**

In [28]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False) # 右边闭合

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

> **pd.cut(ages, bins, labels=group_names)自定义组名**

In [29]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

> **pd.cut(data, 4, precision=2)传入固定数值**

In [30]:
import pandas as pd
import numpy as np
data = np.random.rand(20)
pd.cut(data,4,precision=2) # 保留两位小数

[(0.23, 0.37], (0.37, 0.51], (0.51, 0.65], (0.23, 0.37], (0.51, 0.65], ..., (0.081, 0.23], (0.23, 0.37], (0.51, 0.65], (0.37, 0.51], (0.37, 0.51]]
Length: 20
Categories (4, interval[float64, right]): [(0.081, 0.23] < (0.23, 0.37] < (0.37, 0.51] < (0.51, 0.65]]

> **pd.qcut(df,arg)根据数据的结果等分划分**

In [31]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(-3.066, -0.639], (-3.066, -0.639], (0.598, 3.253], (-0.639, -0.0403], (-3.066, -0.639], ..., (-0.0403, 0.598], (-0.0403, 0.598], (-0.639, -0.0403], (-3.066, -0.639], (-0.639, -0.0403]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.066, -0.639] < (-0.639, -0.0403] < (-0.0403, 0.598] < (0.598, 3.253]]

### **检测和过滤异常值**

In [32]:
import numpy as np 
import pandas as pd
data = pd.DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.011115,0.032001,0.015561,0.035017
std,1.012250,1.047238,0.974967,0.985297
min,-3.263053,-3.640808,-2.889360,-3.973813
25%,-0.654999,-0.653527,-0.613384,-0.597494
50%,0.038262,-0.018157,0.016331,0.092078
75%,0.658113,0.737168,0.673600,0.711102
max,3.690562,3.193914,2.844794,3.175894


In [33]:
col = data[2]
col[np.abs(col)>3] # 过滤数值

Series([], Name: 2, dtype: float64)

In [34]:
data[(np.abs(data) > 3).any(1)] 

,0,1,2,3
25,-3.263053,-0.325337,0.996583,-0.051748
73,-2.470165,0.286102,0.568119,3.175894
144,3.690562,0.992603,-0.100349,0.784877
191,0.879741,3.193914,1.128074,-0.529311
335,3.010760,0.071539,-0.860282,1.593587
355,3.387612,1.810004,-0.389976,-0.015550
505,-0.853336,3.131458,0.292158,-0.716763
516,-2.318364,3.066373,-0.256025,-0.746827
520,2.402419,-3.640808,0.832966,0.468957
544,3.017461,-1.873805,-1.195408,-0.478070


In [35]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.010271,0.032261,0.015561,0.036174
std,1.007842,1.043097,0.974967,0.980077
min,-3.000000,-3.000000,-2.889360,-3.000000
25%,-0.654999,-0.653527,-0.613384,-0.597494
50%,0.038262,-0.018157,0.016331,0.092078
75%,0.658113,0.737168,0.673600,0.711102
max,3.000000,3.000000,2.844794,3.000000


In [36]:
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,1.0,1.0
1,-1.0,-1.0,-1.0,-1.0
2,1.0,-1.0,-1.0,1.0
3,-1.0,-1.0,1.0,-1.0
4,1.0,1.0,-1.0,1.0


### **排列和随机采样**

In [37]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


> **_np.random.permuration()生成序列_**

In [38]:
sampler = np.random.permutation(5)
sampler

array([3, 2, 4, 0, 1])

> **_df.take()使用序列排序_**

In [39]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3
1,4,5,6,7


> **df.sample(n=3)不替换的方式选取**

In [40]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19


In [41]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

3    6
3    6
4    4
3    6
4    4
1    7
2   -1
1    7
0    5
4    4
dtype: int64

### **_计算指标/哑变量_**

In [45]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                       'data1': range(6)})

pd.get_dummies(df['key'])
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                        header=None, names=mnames)

movies[:10]


C:\Users\mjl\AppData\Local\Temp\ipykernel_16008\123407089.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::',


array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [46]:
all_genres = []
for x in movies.genres:
            all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

---
## **字符串操作**


### **字符串对象方法**

> **val.split()拆分字符串**

In [47]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

> **x.strip()去除空字符和换行符**

In [48]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [49]:
first, second, third = pieces
first + '::' +second + '::' +third

'a::b::guido'

In [50]:
'::'.join(pieces)

'a::b::guido'

In [51]:
'guido' in val

True

In [52]:
val.index(',')

1

In [53]:
val.find(':')

-1

In [54]:
val.count(',')

2

In [55]:
val.replace(',','::')

'a::b::  guido'

In [56]:
val.replace(',','')

'ab  guido'

![字符串方法](img/%E5%AD%97%E7%AC%A6%E4%B8%B2%E6%96%B9%E6%B3%95.png)

### **正则表达式**

### **正则表达式**

In [58]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [59]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

> **_data.map，所有字符串和正则表达式方法都能被应用于（传入lambda表达式或其他函数）各个值，但是如果存在NA（null）就会报错。_**

In [60]:
data.str.contains('gmail') # 检查是否包含

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [ ]:
# 也可以使用正则表达式 pip install pattern
import pattern
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

In [ ]:
matches.str.get(1)

In [ ]:
matches.str[0]

In [ ]:
data.str[:5]

![](img/pandas%E5%AD%97%E7%AC%A6%E4%B8%B2%E6%96%B9%E6%B3%95.png)